### Python processing of JSON files to build visualizations using matplotlib

In [8]:
# import libraries
import sys
import os
import re
import json
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.image as mpimg

In [9]:
# path to maps and map image dictionary
PATH_TO_WW2_MAPS = '../week6/cwl-data/maps/ww2/'

MAP_IMAGES = {
    'Aachen': 'aachen.png',
    'Ardennes Forest': 'ardennes_forest.png',
    'Flak Tower': 'flaktower.png',
    'Gibraltar': 'gibraltar.png',
    'London Docks': 'london_docks.png',
    'Sainte Marie du Mont': 'sainte_marie_du_mont.png' ,
    'USS Texas': 'uss_texas.png',
    'Valkyrie': 'valkyrie.png'
}

In [10]:
# list of events
pattern = re.compile(r'structured-2018')
event_list = list(filter(pattern.match, os.listdir("./")))
#event_list

# dictionary of events and list of matches given an event
# {event:[match list]}
dict_event_matches = {}
for i in range(0, len(event_list)):
    dict_event_matches.update({event_list[i]: os.listdir('./{}'.format(event_list[i]))})

In [11]:
# create directory for PNG plots
PATH_TO_PICS = './png_plots/'

if not os.path.exists(PATH_TO_PICS):
    os.makedirs(PATH_TO_PICS)

In [12]:
# checked with ls | wc -l and number of jsons check out
for i in dict_event_matches:
    print(len(dict_event_matches[str(i)]), ': ', i)

508 :  structured-2018-07-29-proleague2
296 :  structured-2018-08-19-champs
506 :  structured-2018-04-08-proleague1
39 :  structured-2018-04-19-relegation
270 :  structured-2018-06-17-anaheim
277 :  structured-2018-03-11-atlanta
272 :  structured-2018-04-22-seattle
164 :  structured-2018-04-01-birmingham
280 :  structured-2018-01-14-neworleans


In [13]:
match_files = []
for EVENT in dict_event_matches:
    for MATCH in dict_event_matches[str(EVENT)]:
        match_files.append('{}/{}'.format(EVENT, MATCH))
    
len(match_files) # 2612 == correct
#match_files[1] # a json file for the match
#os.listdir('./')

2612

In [16]:
# GIGANTIC LOOP for plotting 2612 plots in respective directories

# load match data and save a PNG of match actions

for i in range(0, len(match_files)):
    
    # assess progress of loop
    if i % 100 == 0:
        print(i)
    
    with open(match_files[i], 'r') as match_file:
        
        # get match data
        match = json.load(match_file)

        dirname, figname = match_files[i].split('.')[0].split('/')

        # make new directory for PNGs for a particular event
        newdir = os.path.join(PATH_TO_PICS, dirname)

        if not os.path.exists(newdir):
            os.makedirs(newdir)

        # create name of PNG
        FIGURE_NAME = figname + str('.png')

        # get position data
        death_pos, spawn_pos = {'x': [], 'y': []}, {'x': [], 'y': []}
        events = match['events']
        for evt in events:
            if evt['type'] == 'spawn':
                spawn_pos['x'].append( evt['data']['pos']['x'] )
                spawn_pos['y'].append( evt['data']['pos']['y'] )
            if evt['type'] == 'death':
                death_pos['x'].append( evt['data']['pos']['x'] )
                death_pos['y'].append( evt['data']['pos']['y'] )

        # get map image
        # account for KeyError: '?' (mislabelled maps)
        try:
            map_file_name = PATH_TO_WW2_MAPS + MAP_IMAGES[match['map']]
            map_image = mpimg.imread(map_file_name)
        except KeyError:
            map_image = 'Unknown Location'
         
        
        # create figure
        fig = Figure()
        FigureCanvas(fig)
        ax = fig.add_subplot(111)
        fig.set_figheight(15)
        fig.set_figwidth(15)

        # plot map
        try:
            ax.imshow(map_image)
            ax.set_facecolor((180/255.0, 200/255.0, 220/255.0))
        except TypeError:
            pass
        
        # plot spawns
        ax.scatter(spawn_pos['x'], spawn_pos['y'], color='blue')
        
        # plot deaths
        ax.scatter(death_pos['x'], death_pos['y'], color='red')
        
        # plot spawn-death vectors
        for i in range( len(death_pos['x']) ):
            ax.plot([spawn_pos['x'][i], death_pos['x'][i]], 
                     [spawn_pos['y'][i], death_pos['y'][i]], color='dimgrey')
        # set title
        ax.set_title('{} ({})'.format(match['map'], match['mode']))

        # save as png
        fig.savefig('{}/{}'.format(newdir, FIGURE_NAME))

            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


TypeError: Image data cannot be converted to float

In [7]:

# FIRST MAP IN LOOP ONLY

# load match data and save a PNG of match actions

with open(match_files[0], 'r') as match_file:
    # get match data
    match = json.load(match_file)

    dirname, figname = match_files[0].split('.')[0].split('/')
    
    # make new directory for PNGs for a particular event
    newdir = os.path.join(PATH_TO_PICS, dirname)
    
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    
    FIGURE_NAME = figname + str('.png')

    
    death_pos, spawn_pos = {'x': [], 'y': []}, {'x': [], 'y': []}
    
    # get pos data
    events = match['events']
    for evt in events:
        if evt['type'] == 'spawn':
            spawn_pos['x'].append( evt['data']['pos']['x'] )
            spawn_pos['y'].append( evt['data']['pos']['y'] )
        if evt['type'] == 'death':
            death_pos['x'].append( evt['data']['pos']['x'] )
            death_pos['y'].append( evt['data']['pos']['y'] )

    # get map image
    map_file_name = PATH_TO_WW2_MAPS + MAP_IMAGES[match['map']]
    map_image = mpimg.imread(map_file_name)

    # create figure
    fig = Figure()
    FigureCanvas(fig)
    ax = fig.add_subplot(111)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    # plot map
    ax.imshow(map_image)
    ax.set_facecolor((180/255.0, 200/255.0, 220/255.0))
    # plot spawns
    ax.scatter(spawn_pos['x'], spawn_pos['y'], color='blue')
    # plot deaths
    ax.scatter(death_pos['x'], death_pos['y'], color='red')
    # plot spawn-death vectors
    for i in range( len(death_pos['x']) ):
        ax.plot([spawn_pos['x'][i], death_pos['x'][i]], 
                 [spawn_pos['y'][i], death_pos['y'][i]], color='dimgrey')
    # set title
    ax.set_title('{} ({})'.format(match['map'], match['mode']))
    
    # save as png
    fig.savefig('{}/{}'.format(newdir, FIGURE_NAME))

    
    